# ICT3104 Project Team 12

## Setup
These libraries are required to run the tool.

In [ ]:
#pip installs
!pip install ruamel.yaml
!pip install tqdm
!pip install imageio
!pip install torch
!pip install moviepy
!pip install -U nvidia-cudnn-cu11

In [ ]:
# Import libraries
import os
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual
from IPython.display import display, clear_output, Video
import yaml
import ruamel.yaml
import subprocess
import shutil
import glob
import re
import datetime
import csv

import ipywidgets as widgets
from IPython.display import display, Image, Video, HTML, clear_output
from base64 import b64encode
import os
from os import walk
import torch
from moviepy.editor import *
import moviepy.editor as mpe

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

from tqdm import tqdm
import time

from time import sleep

## Data Exploration
This section lets you select and view the videos from the data/input_files/training_videos folder, allowing you to verify if you have chosen the correct video.
To view the videos, select an option from the dropdown video and wait for the video to be displayed.

In [ ]:
# Define folder path of training videos
main_file_path = f"./ict3104-team12-2023/data/input_files/"

@interact
def select_subfolder(video_folder=os.listdir(main_file_path)):
    video_folder_path = f"{main_file_path}{video_folder}/"

    @interact
    def show_video(file=os.listdir(video_folder_path)):
        if(".mp4" not in file):
            print("Please choose a file with .mp4 format.")
            return
        video_path = f"{video_folder_path}{file}"
        video_display = Video(video_path, width=512, height=512, embed=True)
        display(video_display)

interactive(children=(Dropdown(description='file', options=('6R7QW.mp4', 'CCCUJ.mp4', 'HB9LP.mp4', 'JIR5V.mp4'…

## Inference

In [ ]:
# Create a text input widget for pretrained_model_path
pretrained_model_path_input = widgets.Text(
    value="./checkpoints/stable-diffusion-v1-4",
    description="Pretrained Model Path:",
)

# Read the existing YAML content from the file
yaml_file_path = "./ict3104-team12-2023/configs/pose_sample.yaml"
with open(yaml_file_path, 'r') as file:
    try:
        data = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)
pretrained_model_path = data['pretrained_model_path']
validation_data = data['validation_data']
prompts = validation_data['prompts']

# Create a text input widget for prompts
prompts_input = widgets.Textarea(
    value = "\n".join(str(i) for i in (validation_data['prompts'])),
    description = "Prompts:",
    rows = 4
)

update_config_button = widgets.Button(description="Update Config")
yaml_file_path = "./ict3104-team12-2023/configs/pose_sample.yaml"
output_widget = widgets.Output()

def update_config(button_click):
    updated_path = pretrained_model_path_input.value
    updated_prompts = prompts_input.value.split("\n")

    data['pretrained_model_path'] = updated_path
    data['validation_data']['prompts'] = updated_prompts
    # Set video length to 8
    data['validation_data']['video_length'] = 8

    # Write the updated content back to the file
    with open(yaml_file_path, 'w') as yaml_file:
        yaml.dump(data, yaml_file, sort_keys=False)

    # Display the updated YAML content in the output widget
    with output_widget:
        clear_output()
        print(yaml.dump(data, sort_keys=False))

update_config_button.on_click(update_config)

# Display input fields
display(pretrained_model_path_input)
display(prompts_input)
# Preview
display(update_config_button, output_widget)

Text(value='./checkpoints/stable-diffusion-v1-4', description='Pretrained Model Path:')

Textarea(value='Iron man on the beach\nStormtrooper on the sea\nAstronaut on the beach', description='Prompts:…

Button(description='Update Config', style=ButtonStyle())

Output()

In [ ]:
video_path = "./ict3104-team12-2023/data/input_files/pose_examples/"

video_files = [f for f in os.listdir(video_path) if f.endswith(".mov")]

video_dropdown = widgets.Dropdown(
    options=video_files,
    description='Select a video:',
)

# Variable to store the selected video
selected_video = None
video_file_path = None

# Display the selected video and store the selection in the variable
def display_video(selected):
    global selected_video, video_file_path
    selected_video = selected
    video_file_path = os.path.join(video_path, selected)
    display(Video(video_file_path, width=400, height=300))

    # Update the print statement
    print("Selected video:", selected_video)

interactive(display_video, selected=video_dropdown)

In [ ]:
%cd FollowYourPose
#get all variables needed
#yaml config used path
yaml_config_path = yaml_file_path #YAML path extracted from selection

#skeleton path
initial_skeleton_path = video_file_path #skeleton path extracted from selection
actual_skeleton_path = initial_skeleton_path[0:-4] + ".mp4"

#convert skeleton vid to mp4
if not os.path.isfile(actual_skeleton_path):
    !ffmpeg -i {initial_skeleton_path} {actual_skeleton_path}

#Execute Inference to produce results
result_array = !TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config={yaml_config_path} --skeleton_path={initial_skeleton_path}
#find result path
for i in range(len(result_array),0,-1):
  if "Saved samples to" in result_array[i]:
    result_location = result_array[i].split("to ")
    break

result_path = result_location[1][0:-4] #result path extracted from finished result

In [ ]:
#Build and Combine Files
for (dirpath, dirnames, filenames) in walk(result_path):
    break

#Temporarily store snippet of video here
new_skeleton_path = "./pose_example/Dummy1.mp4" #change this to reference a possible path
edited_video_path = "./pose_example/Dummy.mp4"  #change this to reference a possible path

#result video path
outputFile = "./output/samples/"
#Edit video based on inputs
ffmpeg_extract_subclip(actual_skeleton_path,0,0.544,targetname=edited_video_path) #cut's video
!ffmpeg -i {edited_video_path} -vf "setpts=1.65*PTS" {new_skeleton_path} #slow video down

clear_output(wait=True)

#initialize list to save all result videos
all_Results = []

#display all results
for i in filenames:
    tempOutputPath = outputFile+i[0:-3]+" Combined.mp4"
    #produce superImpose Clip
    result_clip = mpe.VideoFileClip(result_path+"/"+i, target_resolution=(1080,1920))

    overlay_clip = mpe.VideoFileClip(new_skeleton_path, has_mask=True, target_resolution=(1080,1920))
    masked_clip = mpe.vfx.mask_color(overlay_clip,color=[0,0,0])

    superImpose_clip = mpe.CompositeVideoClip([result_clip,masked_clip])
    combined_clip = mpe.clips_array([[result_clip,overlay_clip,superImpose_clip]])
    combined_clip = combined_clip.resize((1680,560))
    combined_clip.write_videofile(
        tempOutputPath,
        remove_temp=True,
        codec="libx264",
        audio_codec="aac",
        threads=6
    )

    all_Results.append(tempOutputPath)

#delete Temporary files
!rm {edited_video_path}
!rm {new_skeleton_path}

In [ ]:
#display results
for i in all_Results:
    #Formatting
    grid = widgets.GridspecLayout(2,1) #(i,j) i= number of columns, j= number of rows

    container = widgets.VBox([grid], layout = widgets.Layout(
        height = '650px',
        width='1690px',
        border='3px solid black',
        justify_content = 'center',
        align_items='center'
    ))

    #show skeleton video
    out = widgets.Output()
    with out:
        display(Video(i,embed=True,html_attributes="loop autoplay"))
    grid[0,0] = out

    #display prompt
    out = widgets.Output()
    with out:
        display(HTML("<center><h1>"+i[0:-14].split("/")[3]+"</h1></center>"))
    grid[1,:] = out

    display(container)

## Training

In [ ]:
# Define source path for pose_train.yaml
pose_train_source_file = "/content/ict3104-team12-2023/training_config_files/pose_train.yaml"

# Define destination path for pose_train.yaml
pose_train_destination_file = "/content/ict3104-team12-2023/FollowYourPose/configs/pose_train.yaml"

# Copy "pose_train.yaml" to the destination directory
shutil.copy(pose_train_source_file, pose_train_destination_file)

This part might not be necessary anymore as client states that changing epoch and batch size is not needed.

However, it will still be needed for ease of directly updating the batch size and max train steps so that we do not need to update manually, since colab GPU is unable to compute high batch size and train steps. (batch_size = 1, max_train_steps = 5)

In [ ]:
# Load the existing YAML configuration from pose_train.yaml
with open('./configs/pose_train.yaml', 'r') as file:
    yaml = ruamel.yaml.YAML()
    data = yaml.load(file)

# Define the generate pose_train.yaml function
def generate_yaml(b):
    # Convert the values in the text widgets to integers (since by default they are treated as strings)
    train_batch_size = int(batch_size_widget.value)
    max_train_steps = int(max_train_steps_widget.value)

    # Update the placeholders with the respective values
    data["train_batch_size"] = train_batch_size
    data["max_train_steps"] = max_train_steps

    # Write the updated data back to the pose_train.yaml file
    with open('./configs/pose_train.yaml', 'w') as file:
        yaml.dump(data, file)

# Create the model name, batch size, and max train steps text widgets
model_name_widget = widgets.Text(description='Model Name:')
batch_size_widget = widgets.Text(description='Batch Size:')
max_train_steps_widget = widgets.Text(description='Max Train Steps')

# Create a button to trigger the update
update_button = widgets.Button(description='Update')
update_button.on_click(generate_yaml)

# Display widgets
display(model_name_widget)
display(batch_size_widget)
display(max_train_steps_widget)
display(update_button)

In [ ]:
# Define source paths for hdvila, train_followyourpose
hdvila_source_file = "/content/ict3104-team12-2023/training_config_files/hdvila.py"
train_followyourpose_source_file = "/content/ict3104-team12-2023/training_config_files/train_followyourpose.py"

# Define destination paths for hdvila, train_followyourpose
hdvila_destination_file = "/content/ict3104-team12-2023/FollowYourPose/followyourpose/data/hdvila.py"
train_followyourpose_destination_file = "/content/ict3104-team12-2023/FollowYourPose/train_followyourpose.py"

# Copy "hdvila.py" to the destination directory
shutil.copy(hdvila_source_file, hdvila_destination_file)

# Copy "train_followyourpose" to the destination directory
shutil.copy(train_followyourpose_source_file, train_followyourpose_destination_file)

# Define source and destination paths for pose_example skeleton videos
#pose_examples_source_directory = '/content/ict3104-team12-2023/data/input_files/pose_examples'
pose_examples_source_directory = '/content/ict3104-team12-2023/data/input_files/training_videos'
pose_examples_destination_directory = '/content/ict3104-team12-2023/FollowYourPose/followyourpose/data'

# Filter out hidden files and count the number of video files
video_files = [f for f in os.listdir(pose_examples_source_directory) if f.endswith('.mp4')]
num_videos = len(video_files)

# Create the part_id folders based on the number of videos
for part_id in range(1, num_videos + 1):
    part_folder = os.path.join(pose_examples_destination_directory, f'part_{part_id}')
    os.makedirs(part_folder, exist_ok=True)

# Distribute each video into a separate video_clips folder with an ascending index
for i, video_file in enumerate(video_files[:10]):
    part_id = i + 1
    part_folder = os.path.join(pose_examples_destination_directory, f'part_{part_id}')
    video_clips_folder = os.path.join(part_folder, 'video_clips')
    os.makedirs(video_clips_folder, exist_ok=True)

    source_path = os.path.join(pose_examples_source_directory, video_file)
    video_count = 1

    while True:
        destination_path = os.path.join(video_clips_folder, f'{video_count:04}.mp4')
        if not os.path.exists(destination_path):
            break
        video_count += 1

    shutil.copy(source_path, destination_path)

# Create the CSV file
csv_filename = 'caption_rm2048_train.csv'
csv_path = os.path.join(pose_examples_destination_directory, csv_filename)

# Create and open the CSV file in write mode
with open(csv_path, 'w', newline='') as csv_file:
    fieldnames = ['part_id', 'clip_id', 'caption']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Iterate through the part folders and video clips
    for part_id in range(1, num_videos + 1):
        part_folder = os.path.join(pose_examples_destination_directory, f'part_{part_id}')
        video_clips_folder = os.path.join(part_folder, 'video_clips')

        for clip_id, video_file in enumerate(os.listdir(video_clips_folder), start=1):
            # Create the caption by concatenating part_id and clip_id
            caption = f'{part_id}_{clip_id:04}'
            clip_id = f'{clip_id:04}'

            # Write the data to the CSV file
            writer.writerow({'part_id': part_id, 'clip_id': clip_id, 'caption': caption})

print(f'CSV file "{csv_filename}" has been created at: {csv_path}')

In [ ]:
# Define the CSV path
csv_path = '/content/ict3104-team12-2023/FollowYourPose/followyourpose/data/caption_rm2048_train.csv'

# Define the base path for video clips
base_path = "/content/ict3104-team12-2023/FollowYourPose/followyourpose/data"

# Create an empty dictionary to store config.yaml file paths
config_files = {}

def get_part_id(directory_name):
    # Extract the part ID from the directory name
    return directory_name.replace("part_", "")

def iterate_video_clips(base_path, destination_folder, csv_data, config_files):
    for part_dir in os.listdir(base_path):
        if part_dir.startswith("part_"):
            part_path = os.path.join(base_path, part_dir, "video_clips")
            part_id = get_part_id(part_dir)

            if os.path.exists(part_path):
                clip_counter = 0

                for clip_file in os.listdir(part_path):
                    # Use the 'clip_id' from CSV data directly
                    clip_id = clip_file.split('.')[0]

                    # Check if the clip_id (file name) exists in the CSV data
                    for row in csv_data:
                        if row['clip_id'] == clip_id and row['part_id'] == part_id:
                            video_info = row

                            # Construct the video path based on the clip_file and the base path
                            video_path = os.path.join(part_path, clip_file)

                            if os.path.exists(video_path):
                                clip_counter += 1
                                clip_id_with_zeros = str(clip_counter).zfill(4)

                                print(f"Part ID: {part_id}")
                                print(f"Clip ID: {clip_id_with_zeros}")
                                print(f"Found video info: {video_info}")
                                print(f"Video path: {video_path}")
                                print(f"Processing video for Clip ID: {clip_id_with_zeros}")

                                # Load the pose_train.yaml
                                with open('/content/ict3104-team12-2023/FollowYourPose/configs/pose_train.yaml', 'r') as config_file:
                                    config = yaml.safe_load(config_file)

                                # Update the skeleton_path in the config dictionary
                                config['skeleton_path'] = f"./followyourpose/data/part_{part_id}/video_clips/{clip_id_with_zeros}.mp4"

                                # Save the updated config to a new file for each clip
                                config_file_path = os.path.join(destination_folder, f"config_{clip_id_with_zeros}_{part_id}.yaml")
                                with open(config_file_path, 'w') as new_config_file:
                                    yaml.dump(config, new_config_file, default_flow_style=False)

                                # Execute the accelerate command on the video
                                command = f"TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch train_followyourpose.py --config='{config_file_path}' --skeleton_path='{video_path}'"
                                result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                                print(result.stdout.decode())
                                print(result.stderr.decode())
                                print(f"Finished processing video: {video_path}")

                                # Copy the entire output folder to the destination folder
                                source_output_folder = "/content/ict3104-team12-2023/FollowYourPose/output"
                                destination_output_folder = os.path.join(destination_folder, f"output_{clip_id_with_zeros}_{part_id}")
                                shutil.copytree(source_output_folder, destination_output_folder)

                                if result.returncode == 0:
                                    print(f"Finished processing video for Clip ID: {clip_id_with_zeros}")
                                else:
                                    print(f"Error processing video for Clip ID: {clip_id_with_zeros}")
                            else:
                                print(f"Video path not found for Clip ID: {clip_id_with_zeros}")

# Define the destination folder for config.yaml files
destination_folder = '/content/ict3104-team12-2023/data/models/trained_model'

# Load CSV data into a list of dictionaries
csv_data = []
with open(csv_path, 'r', newline='') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        csv_data.append(row)

# Process the videos and generate the config.yaml files
iterate_video_clips(base_path, destination_folder, csv_data, config_files)

In [ ]:

# Create a progress bar widget
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
)

# Display the progress bar
display(progress_bar)

# Define the number of iterations in your for loop
total_iterations = 100

# Use tqdm to create a progress bar for the for loop
for i in tqdm(range(total_iterations), desc="Processing"):
    # Simulate some work being done
    time.sleep(0.1)

    # Update the progress bar
    progress_bar.value = (i / total_iterations) * 100

# Remove the progress bar when the loop is complete
progress_bar.close()


In [ ]:
# Saving most optimal model config

# Change to the source directory
source_dir = "/content/ict3104-team12-2023/data/models/trained_model"
os.chdir(source_dir)

# Find all the subdirectories in the source directory
subdirectories = [d for d in os.listdir() if os.path.isdir(d)]

if subdirectories:
    # Sort the subdirectories by modification time to get the latest one
    latest_subdirectory = max(subdirectories, key=lambda d: os.path.getmtime(os.path.join(source_dir, d)))

    # Define the destination directory
    destination_dir = "/content/ict3104-team12-2023/data/models/final_model"

    # Check if the destination directory exists, create it if it doesn't
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Get the current timestamp in the format yyyymmdd_hhmmss
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # Create the new directory name with the timestamp
    new_directory_name = f"{latest_subdirectory}_{timestamp}"

    # Combine the destination directory and the new directory name
    new_destination_directory = os.path.join(destination_dir, new_directory_name)

    # Move the latest subdirectory to the destination directory with the new directory name
    shutil.move(os.path.join(source_dir, latest_subdirectory), new_destination_directory)
    print(f"Latest output folder renamed and moved to '{new_destination_directory}'.")
else:
    print("No output folders found in the source directory.")

## Testing

In [ ]:
# For dataset subfolders
folder_path = "./ict3104-team12-2023/data/"

subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

dropdown = widgets.Dropdown(
    options=subfolders,
    description='Select a subfolder:',
)

# Variable to store the selected subfolder
selected_subfolder = None

# Display the selected subfolder
def display_selected_subfolder(selected):
    global selected_subfolder
    selected_subfolder = selected
    clear_output()
    print(f"Selected subfolder: {selected_subfolder}")

interactive(display_selected_subfolder, selected=dropdown)

In [ ]:
# For model / config file selection
folder_path = "./ict3104-team12-2023/data/models/"

yaml_files = [f for f in os.listdir(folder_path) if f.endswith(".yaml")]
all_files = os.listdir(folder_path)

dropdown = widgets.Dropdown(
    options=all_files,
    description='Select a model:',
)

# Variable to store the selected file
selected_file = None

# Display the selected file
def display_selected_file(selected):
    global selected_file
    selected_file = selected
    clear_output()
    print(f"Selected file: {selected_file}")

interactive(display_selected_file, selected=dropdown)